<a href="https://colab.research.google.com/github/ikuldeep1/Llama-3.2-11B-Vision-Colab/blob/main/Llama_3_2_11B_Vision_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitsandbytes accelerate -q

In [2]:
from huggingface_hub import login

login(token="your_huggingface_token_here")

In [ ]:
import torch
from transformers import AutoProcessor, MllamaForConditionalGeneration
from transformers import BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_id)


In [ ]:
from PIL import Image

image = Image.open('/content/images/page_4.png')

def chat_with_mllm(model, processor, prompt, image, messages=[]):
    input_text = processor.apply_chat_template(
        messages + [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": prompt}]}],
        add_generation_prompt=True,
        tokenize=False
    )

    inputs = processor(image, input_text, return_tensors="pt").to(model.device)

    output = model.generate(**inputs, max_new_tokens=512)
    response = processor.batch_decode(output, skip_special_tokens=True)[0]

    messages.append({"role": "assistant", "content": [{"type": "text", "text": response}]})
    return response, messages


In [ ]:
prompt = "Extract all the text from the image with layout. If the image has flow diagram then extract the flow diagram and represent in steps. I need to pass this information the LLM so extract the text in a format that LLM can easily understand the semantic context. Do not add summary or explain anything. Just extract all the text in proper layout."
response, messages = chat_with_mllm(model, processor, prompt, image)
print(response)